In [1]:
!pip install lightfm scikit-optimize

In [2]:
# import dependent libraries
import pandas as pd
from scipy.sparse import csr_matrix
import numpy as np

import matplotlib.pyplot as plt
from matplotlib.gridspec import GridSpec
import seaborn as sns
%matplotlib inline

from lightfm.cross_validation import random_train_test_split
from lightfm.evaluation import auc_score, precision_at_k, recall_at_k
from lightfm import LightFM
from skopt import forest_minimize

/opt/anaconda3/lib/python3.8/site-packages/lightfm/_lightfm_fast.py:9: UserWarning: LightFM was compiled without OpenMP support. Only a single thread will be used.
  warnings.warn(


In [3]:
df_res = pd.read_csv('/Users/zeenath/Desktop/british_final3.csv')

df_res

,Unnamed: 0,Unnamed: 0.1,Name,Rating,No_of_Reviews,Latitude,Longitude,day_a,day_a1,time_a,...,Dinner,Drinks,Lunch,None_y,After-hours,Breakfast,Brunch,Dinner,Drinks,Lunch
0,0,0,Freshii,3.5,9.0,49.036045,-122.293106,1.0,5.0,7.0,...,0,0,0,0,0,0,0,0,0,1
1,1,1,Pantry Family Restaurant,4.0,10.0,49.049360,-122.312770,0.0,4.0,6.3,...,0,0,0,0,0,0,0,0,0,1
2,2,2,Townhall Public House Abbotsford,4.0,123.0,49.047710,-122.291150,0.0,NaN,10.0,...,0,0,0,1,0,0,0,0,0,0
3,3,3,Golden Dragon Restaurant,3.5,39.0,49.048800,-122.295150,0.0,1.0,11.3,...,0,0,1,0,0,0,0,1,0,0
4,4,4,Townline Taphouse & Grill,1.0,1.0,49.068900,-122.358580,NaN,NaN,NaN,...,0,0,0,1,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9430,33218,33218,Hopkins Harvest & The Hot Spot,4.5,91.0,50.468094,-115.980450,1.0,2.0,9.0,...,0,0,0,1,0,0,0,0,0,0
9431,33221,33221,Edible Acres Cafe,5.0,7.0,50.467415,-115.978590,2.0,6.0,9.0,...,0,0,0,0,0,0,0,0,0,1
9432,33279,33279,Two Scoop Steve,4.5,27.0,49.080708,-116.103870,0.0,4.0,10.0,...,0,0,1,0,0,0,0,1,0,0
9433,33280,33280,Smoke'N'Jugz Roadhouse,4.5,19.0,49.085630,-116.089920,3.0,0.0,12.0,...,1,0,0,0,0,0,0,0,0,1


In [4]:
df_res.head(2)

,Unnamed: 0,Unnamed: 0.1,Name,Rating,No_of_Reviews,Latitude,Longitude,day_a,day_a1,time_a,...,Dinner,Drinks,Lunch,None_y,After-hours,Breakfast,Brunch,Dinner,Drinks,Lunch
0,0,0,Freshii,3.5,9.0,49.036045,-122.293106,1.0,5.0,7.0,...,0,0,0,0,0,0,0,0,0,1
1,1,1,Pantry Family Restaurant,4.0,10.0,49.049360,-122.312770,0.0,4.0,6.3,...,0,0,0,0,0,0,0,0,0,1


In [5]:
df_res.drop(df_res.columns[7:40], axis='columns', inplace=True)
df_res


,Unnamed: 0,Unnamed: 0.1,Name,Rating,No_of_Reviews,Latitude,Longitude,res_id,City_encode,Price_code,...,Dinner,Drinks,Lunch,None_y,After-hours,Breakfast,Brunch,Dinner,Drinks,Lunch
0,0,0,Freshii,3.5,9.0,49.036045,-122.293106,0,3,1,...,0,0,0,0,0,0,0,0,0,1
1,1,1,Pantry Family Restaurant,4.0,10.0,49.049360,-122.312770,1,3,0,...,0,0,0,0,0,0,0,0,0,1
2,2,2,Townhall Public House Abbotsford,4.0,123.0,49.047710,-122.291150,2,3,0,...,0,0,0,1,0,0,0,0,0,0
3,3,3,Golden Dragon Restaurant,3.5,39.0,49.048800,-122.295150,3,3,0,...,0,0,1,0,0,0,0,1,0,0
4,4,4,Townline Taphouse & Grill,1.0,1.0,49.068900,-122.358580,4,3,3,...,0,0,0,1,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9430,33218,33218,Hopkins Harvest & The Hot Spot,4.5,91.0,50.468094,-115.980450,33218,224,0,...,0,0,0,1,0,0,0,0,0,0
9431,33221,33221,Edible Acres Cafe,5.0,7.0,50.467415,-115.978590,33221,224,0,...,0,0,0,0,0,0,0,0,0,1
9432,33279,33279,Two Scoop Steve,4.5,27.0,49.080708,-116.103870,33279,226,1,...,0,0,1,0,0,0,0,1,0,0
9433,33280,33280,Smoke'N'Jugz Roadhouse,4.5,19.0,49.085630,-116.089920,33280,226,0,...,1,0,0,0,0,0,0,0,0,1


In [6]:
df_res

,Unnamed: 0,Unnamed: 0.1,Name,Rating,No_of_Reviews,Latitude,Longitude,res_id,City_encode,Price_code,...,Dinner,Drinks,Lunch,None_y,After-hours,Breakfast,Brunch,Dinner,Drinks,Lunch
0,0,0,Freshii,3.5,9.0,49.036045,-122.293106,0,3,1,...,0,0,0,0,0,0,0,0,0,1
1,1,1,Pantry Family Restaurant,4.0,10.0,49.049360,-122.312770,1,3,0,...,0,0,0,0,0,0,0,0,0,1
2,2,2,Townhall Public House Abbotsford,4.0,123.0,49.047710,-122.291150,2,3,0,...,0,0,0,1,0,0,0,0,0,0
3,3,3,Golden Dragon Restaurant,3.5,39.0,49.048800,-122.295150,3,3,0,...,0,0,1,0,0,0,0,1,0,0
4,4,4,Townline Taphouse & Grill,1.0,1.0,49.068900,-122.358580,4,3,3,...,0,0,0,1,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9430,33218,33218,Hopkins Harvest & The Hot Spot,4.5,91.0,50.468094,-115.980450,33218,224,0,...,0,0,0,1,0,0,0,0,0,0
9431,33221,33221,Edible Acres Cafe,5.0,7.0,50.467415,-115.978590,33221,224,0,...,0,0,0,0,0,0,0,0,0,1
9432,33279,33279,Two Scoop Steve,4.5,27.0,49.080708,-116.103870,33279,226,1,...,0,0,1,0,0,0,0,1,0,0
9433,33280,33280,Smoke'N'Jugz Roadhouse,4.5,19.0,49.085630,-116.089920,33280,226,0,...,1,0,0,0,0,0,0,0,0,1


In [7]:
df_res.columns

Index(['Unnamed: 0', 'Unnamed: 0.1', 'Name', 'Rating', 'No_of_Reviews',
       'Latitude', 'Longitude', 'res_id', 'City_encode', 'Price_code',
       ...
       'Dinner', 'Drinks', 'Lunch', 'None_y', ' After-hours', ' Breakfast',
       ' Brunch', ' Dinner', ' Drinks', ' Lunch'],
      dtype='object', length=9944)

In [8]:
df_res.columns.unique()

Index(['Unnamed: 0', 'Unnamed: 0.1', 'Name', 'Rating', 'No_of_Reviews',
       'Latitude', 'Longitude', 'res_id', 'City_encode', 'Price_code',
       ...
       'Dinner', 'Drinks', 'Lunch', 'None_y', ' After-hours', ' Breakfast',
       ' Brunch', ' Dinner', ' Drinks', ' Lunch'],
      dtype='object', length=9944)

In [9]:
df_res.describe()

,Unnamed: 0,Unnamed: 0.1,Rating,No_of_Reviews,Latitude,Longitude,res_id,City_encode,Price_code,Est_code,...,Dinner,Drinks,Lunch,None_y,After-hours,Breakfast,Brunch,Dinner,Drinks,Lunch
count,9435.000000,9435.000000,9435.000000,9435.000000,9435.000000,9435.000000,9435.000000,9435.000000,9435.000000,9435.000000,...,9435.000000,9435.000000,9435.000000,9435.000000,9435.000000,9435.000000,9435.000000,9435.000000,9435.000000,9435.000000
mean,11709.736089,11709.736089,4.066402,91.358029,49.631154,-122.384501,11709.736089,105.180074,0.777318,4.859247,...,0.113831,0.006253,0.376471,0.346794,0.008267,0.024801,0.016216,0.338209,0.010493,0.175305
std,9177.821662,9177.821662,0.597292,210.548357,1.274150,2.306661,9177.821662,76.973878,1.131353,0.578122,...,0.317623,0.078834,0.484526,0.475975,0.090552,0.155527,0.126313,0.473125,0.101901,0.380248
min,0.000000,0.000000,0.000000,1.000000,45.511170,-134.920150,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,3394.500000,3394.500000,4.000000,9.000000,49.164097,-123.159280,3394.500000,36.000000,0.000000,5.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,10418.000000,10418.000000,4.000000,27.000000,49.269000,-123.072840,10418.000000,72.000000,0.000000,5.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,19225.500000,19225.500000,4.500000,80.000000,49.563378,-122.337492,19225.500000,182.000000,1.000000,5.000000,...,0.000000,0.000000,1.000000,1.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000
max,33281.000000,33281.000000,5.000000,4507.000000,59.772747,-73.563010,33281.000000,228.000000,3.000000,6.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [13]:
df = pd.DataFrame(df_res) 

In [15]:
df.Drinks.unique() 

array([0, 1])

In [16]:
for Drinks in df:
    print(df[Drinks].unique())


[    0     1     2 ... 33279 33280 33281]
[    0     1     2 ... 33279 33280 33281]
['Freshii' 'Pantry Family Restaurant' 'Townhall Public House Abbotsford'
 ... 'Two Scoop Steve' "Smoke'N'Jugz Roadhouse" 'Woss Vegas Cafe']
[3.5 4.  1.  3.  4.5 1.5 2.5 5.  2.  0. ]
[9.000e+00 1.000e+01 1.230e+02 3.900e+01 1.000e+00 7.600e+01 8.000e+01
 1.800e+01 3.400e+01 1.100e+01 5.800e+01 6.000e+00 1.990e+02 1.400e+01
 4.700e+01 4.600e+01 3.500e+01 5.000e+00 2.200e+01 3.200e+01 4.000e+00
 5.100e+01 3.420e+02 3.610e+02 2.700e+01 1.900e+01 2.000e+00 3.100e+01
 7.000e+00 2.100e+01 1.700e+01 8.000e+00 8.700e+01 1.300e+01 1.430e+02
 5.200e+01 2.300e+01 2.400e+01 2.640e+02 3.700e+01 6.500e+01 1.600e+01
 4.180e+02 2.600e+01 5.600e+01 2.620e+02 3.000e+01 2.240e+02 7.300e+01
 2.250e+02 3.600e+01 1.110e+02 7.400e+01 8.600e+01 9.500e+01 3.000e+00
 2.500e+01 4.800e+01 2.000e+01 2.540e+02 2.900e+01 5.890e+02 1.180e+02
 7.500e+01 9.200e+01 1.340e+02 1.010e+02 6.940e+02 1.380e+02 1.500e+01
 1.360e+02 5.400e+01 8.1

[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1

[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1

[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1

[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1

[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1

[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1

[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[1 0]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[1 0]


In [17]:
df

,Unnamed: 0,Unnamed: 0.1,Name,Rating,No_of_Reviews,Latitude,Longitude,res_id,City_encode,Price_code,...,Dinner,Drinks,Lunch,None_y,After-hours,Breakfast,Brunch,Dinner,Drinks,Lunch
0,0,0,Freshii,3.5,9.0,49.036045,-122.293106,0,3,1,...,0,0,0,0,0,0,0,0,0,1
1,1,1,Pantry Family Restaurant,4.0,10.0,49.049360,-122.312770,1,3,0,...,0,0,0,0,0,0,0,0,0,1
2,2,2,Townhall Public House Abbotsford,4.0,123.0,49.047710,-122.291150,2,3,0,...,0,0,0,1,0,0,0,0,0,0
3,3,3,Golden Dragon Restaurant,3.5,39.0,49.048800,-122.295150,3,3,0,...,0,0,1,0,0,0,0,1,0,0
4,4,4,Townline Taphouse & Grill,1.0,1.0,49.068900,-122.358580,4,3,3,...,0,0,0,1,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9430,33218,33218,Hopkins Harvest & The Hot Spot,4.5,91.0,50.468094,-115.980450,33218,224,0,...,0,0,0,1,0,0,0,0,0,0
9431,33221,33221,Edible Acres Cafe,5.0,7.0,50.467415,-115.978590,33221,224,0,...,0,0,0,0,0,0,0,0,0,1
9432,33279,33279,Two Scoop Steve,4.5,27.0,49.080708,-116.103870,33279,226,1,...,0,0,1,0,0,0,0,1,0,0
9433,33280,33280,Smoke'N'Jugz Roadhouse,4.5,19.0,49.085630,-116.089920,33280,226,0,...,1,0,0,0,0,0,0,0,0,1


In [18]:
df_dict = dict(zip([i for i in df.columns] , [pd.DataFrame(df[i].unique(), columns=[i]) for i in df.columns]))


In [19]:
df_dict[Drinks] 

,Lunch
0,1
1,0
